In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data

In [3]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        super(RNN, self).__init__()        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=False
        )
        self.out = nn.Linear(hidden_size, output_size)        
        self.hidden = self.init_hidden()
        
    def init_hidden(self, x=None):
        if x == None:
            return (Variable(torch.zeros(self.num_layers, 1, self.hidden_size)),
                    Variable(torch.zeros(self.num_layers, 1, self.hidden_size)))
        else:
            return (Variable(x[0].data),Variable(x[1].data))
        
    def forward(self, x):
        lstm_out, self.hidden_out = self.lstm(x, self.hidden)
        output = self.out(lstm_out.view(len(x), -1))
        self.hidden = self.init_hidden(self.hidden_out)
        return output

In [4]:
INPUT_SIZE = 22
HIDDEN_SIZE = 100
NUM_LAYERS = 2
NUM_EPOCHS = 15
LEARNING_RATE = 0.001
rnn = RNN(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, 3)
rnn.load_state_dict(torch.load('rnn_params.pt'))

In [5]:
dataset = pd.read_csv('train_data/f-speedway.csv', dtype=float)
X = dataset.iloc[0, 3:].values

In [6]:
X = torch.from_numpy(X).float()

In [14]:
params = Variable(X.view(-1, 1, INPUT_SIZE))
o = rnn(params)

In [15]:
o

Variable containing:
 0.6494 -0.1557  0.0669
[torch.FloatTensor of size 1x3]